In [17]:
import pandas as pd
import numpy as np
import csv
colNames = ['class', 'ra'
            , 'dec', 'u', 'g', 'r', 'i', 'z', 'nuv_mag']

dataSet = pd.read_csv('../GALEX_data-extended-feats.csv', usecols = colNames)
layers = [8, 64, 3]
dataSet = pd.DataFrame(dataSet)

In [18]:
def sig(z):
    return (1)/(1 + np.exp(-z))

def sig_prime(z):
    return sig(z)*(1-sig(z))

def tanH(z):
    return np.tanh(z)

def tanH_prime(z):
    return 1 - ((tanH(z))**2)

In [19]:
weightsHidden = np.array(np.random.randn(layers[1], layers[0]))*(np.sqrt(2/(layers[1] + layers[0])))
biasHidden = np.zeros(layers[1])
weightsOut = np.array(np.random.randn(layers[2], layers[1]))*(np.sqrt(2/(layers[1] + layers[2])))
biasOut = np.zeros(layers[2])
testDf = dataSet.sample(frac = 0.2, random_state = 3)

In [22]:
def expec(inp):
    if inp == 0:
        return np.array([1, 0, 0])
    elif inp == 1:
        return np.array([0, 1, 0])
    else:
        return np.array([0, 0, 1])

def feedFor(trainDf, row, weightsHidden, biasHidden, weightsOut, biasOut):
    x = np.array(trainDf.iloc[row, 1:])
    tempDot = np.array(np.dot(weightsHidden, x.T)) # 20 x 1
    hiddenLI = np.add(tempDot, biasHidden) # 1 x 20
    hiddenAct = tanH(hiddenLI) # 1 x 20
    tempDot2 = np.dot(weightsOut, hiddenAct)
    outputLI = np.add(tempDot2, biasOut.T)
    output = tanH(outputLI)
#     print(output)
    return x, output, hiddenAct

def backProp(trainDf, layers, x, row, output, hiddenAct, weightsHidden, biasHidden, weightsOut, biasOut, eta):
    actClass = int(trainDf.iloc[row, 0])
    expected = expec(actClass)
    error = expected - output
    slopeOutput = tanH_prime(np.array(output))
    slopeHL = tanH_prime(np.array(hiddenAct))
    deltaO = error*slopeOutput
    errHL = np.dot(weightsOut.T, deltaO)
    deltaHL = errHL*slopeHL
    
    deltaO = np.reshape(deltaO, (3, 1))
    hiddenAct = np.reshape(hiddenAct, (layers[1], 1))
    tempDot = np.dot(deltaO, hiddenAct.T)*(eta) # 3 x 20
    weightsOut += tempDot
    
    deltaHL = np.reshape(deltaHL, (layers[1], 1))
    x = np.reshape(x, (8, 1))
    tempDot2 = np.dot(deltaHL, x.T)*(eta)
    weightsHidden += tempDot2
    biasHidden += np.sum(deltaHL, axis = 0)*(eta)
    biasOut += np.sum(deltaO, axis = 0)*(eta)


def runNN(layers, trainDf, weightsHidden, biasHidden, weightsOut, biasOut, eta):
    for row in range(len(trainDf)):
        x, output, hiddenAct = feedFor(trainDf, row, weightsHidden, biasHidden, weightsOut, biasOut)
        backProp(trainDf, layers, x, row, output, hiddenAct, weightsHidden, biasHidden, weightsOut, biasOut, eta)

def accu():
    correct = 0
    for row in range(len(testDf)):
        actClass = int(testDf.iloc[row, 0])
        x = np.array(testDf.iloc[row, 1:])
        tempDot = np.array(np.dot(weightsHidden, x.T)) # 20 x 1
        hiddenLI = np.add(tempDot, biasHidden) # 1 x 20
        hiddenAct = tanH(hiddenLI) # 1 x 20
        tempDot2 = np.dot(weightsOut, hiddenAct)
        outputLI = np.add(tempDot2, biasOut.T)
        output = tanH(outputLI)
        expected = expec(actClass)
        if np.argmax(expected) == np.argmax(output):
            #print("Instance classified as: ", np.argmax(output))
            correct += 1
        else:
            pass
            #print("Instance misclassified as: ", np.argmax(output))
        #print(output)
    return correct/len(testDf)

filename = "accuracy_0.005_32.csv"
fields = ['Training Data', 'Accuracy']
with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields) 
    eta = 0.005
    lenDf = 0
    for i in range(10):
        trainDf = dataSet.sample(frac = 0.8)
        lenDf += len(trainDf)
        runNN(layers, trainDf, weightsHidden, biasHidden, weightsOut, biasOut, eta)
        acc = accu()
        newRow = [lenDf, acc]
        csvwriter.writerow(newRow)
        print(i, acc)



0 0.851279000594884
1 0.8488994646044021
2 0.8506841165972635
3 0.7977394408090422
4 0.8530636525877454
5 0.8274836406900654
6 0.8524687685901249
7 0.8465199286139203
8 0.851279000594884
9 0.8352171326591314


In [16]:
np.save("wH", weightsHidden)
np.save("bH", biasHidden)
np.save("wO", weightsOut)
np.save("bO", biasOut)